<a href="https://colab.research.google.com/github/clionelove123/Machine_Learning/blob/main/ML_Chap_09_TSP_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
def makeTSP(nCities):
	positions = 2*np.random.rand(nCities,2)-1;
	distances = np.zeros((nCities,nCities))

	for i in range(nCities):
		for j in range(i+1,nCities):
			distances[i,j] = np.sqrt((positions[i,0] - positions[j,0])**2 + (positions[i,1] - positions[j,1])**2);
			distances[j,i] = distances[i,j];

	return distances

In [ ]:
def exhaustive(distances):
	nCities = np.shape(distances)[0]

	cityOrder = np.arange(nCities)

	distanceTravelled = 0
	for i in range(nCities-1):
		distanceTravelled += distances[cityOrder[i],cityOrder[i+1]]
	distanceTravelled += distances[cityOrder[nCities-1],0]

	for newOrder in permutation(range(nCities)):
		possibleDistanceTravelled = 0
		for i in range(nCities-1):
			possibleDistanceTravelled += distances[newOrder[i],newOrder[i+1]]
		possibleDistanceTravelled += distances[newOrder[nCities-1],0]

		if possibleDistanceTravelled < distanceTravelled:
			distanceTravelled = possibleDistanceTravelled
			cityOrder = newOrder

	return cityOrder, distanceTravelled

In [ ]:
def permutation(order):
	order = tuple(order)
	if len(order)==1:
		yield order
	else:
		for i in range(len(order)):
			rest = order[:i] + order[i+1:]
			move = (order[i],)
			for smaller in permutation(rest):
				yield move + smaller

In [ ]:
#def greedy(distances):
#	nCities = np.shape(distances)[0]
#	distanceTravelled = 0

	# Need a version of the matrix we can trash
#	dist = distances.copy()

#	cityOrder = np.zeros(nCities)
#	cityOrder[0] = np.random.randint(nCities)
#	dist[:,cityOrder[0]] = np.Inf

#	for i in range(nCities-1):
#		cityOrder[i+1] = np.argmin(dist[cityOrder[i],:])
#		distanceTravelled  += dist[cityOrder[i],cityOrder[i+1]]
		# Now exclude the chance of travelling to that city again
#		dist[:,cityOrder[i+1]] = np.Inf

	# Now return to the original city
#	distanceTravelled += distances[cityOrder[nCities-1],0]

#	return cityOrder, distanceTravelled

In [ ]:
def greedy(distances):
    nCities = np.shape(distances)[0]
    distanceTravelled = 0

    # Need a version of the matrix we can trash
    dist = distances.copy()

    cityOrder = np.zeros(nCities, dtype=int)
    cityOrder[0] = np.random.randint(nCities)
    dist[:, cityOrder[0]] = np.Inf

    for i in range(nCities - 1):
        cityOrder[i + 1] = np.argmin(dist[cityOrder[i], :])
        distanceTravelled += dist[cityOrder[i], cityOrder[i + 1]]
        # Now exclude the chance of traveling to that city again
        dist[:, cityOrder[i + 1]] = np.Inf

    # Now return to the original city
    distanceTravelled += distances[cityOrder[nCities - 1], 0]

    return cityOrder, distanceTravelled

In [ ]:
def hillClimbing(distances):

	nCities = np.shape(distances)[0]

	cityOrder = np.arange(nCities)
	np.random.shuffle(cityOrder)

	distanceTravelled = 0
	for i in range(nCities-1):
		distanceTravelled += distances[cityOrder[i],cityOrder[i+1]]
	distanceTravelled += distances[cityOrder[nCities-1],0]

	for i in range(1000):
		# Choose cities to swap
		city1 = np.random.randint(nCities)
		city2 = np.random.randint(nCities)

		if city1 != city2:
			# Reorder the set of cities
			possibleCityOrder = cityOrder.copy()
			possibleCityOrder = np.where(possibleCityOrder==city1,-1,possibleCityOrder)
			possibleCityOrder = np.where(possibleCityOrder==city2,city1,possibleCityOrder)
			possibleCityOrder = np.where(possibleCityOrder==-1,city2,possibleCityOrder)

			# Work out the new distances
			# This can be done more efficiently
			newDistanceTravelled = 0
			for j in range(nCities-1):
				newDistanceTravelled += distances[possibleCityOrder[j],possibleCityOrder[j+1]]
			distanceTravelled += distances[cityOrder[nCities-1],0]

			if newDistanceTravelled < distanceTravelled:
				distanceTravelled = newDistanceTravelled
				cityOrder = possibleCityOrder

	return cityOrder, distanceTravelled

In [ ]:
def simulatedAnnealing(distances):

	nCities = np.shape(distances)[0]

	cityOrder = np.arange(nCities)
	np.random.shuffle(cityOrder)

	distanceTravelled = 0
	for i in range(nCities-1):
		distanceTravelled += distances[cityOrder[i],cityOrder[i+1]]
	distanceTravelled += distances[cityOrder[nCities-1],0]

	T = 500
	c = 0.8
	nTests = 10

	while T>1:
		for i in range(nTests):
			# Choose cities to swap
			city1 = np.random.randint(nCities)
			city2 = np.random.randint(nCities)

			if city1 != city2:
				# Reorder the set of cities
				possibleCityOrder = cityOrder.copy()
				possibleCityOrder = np.where(possibleCityOrder==city1,-1,possibleCityOrder)
				possibleCityOrder = np.where(possibleCityOrder==city2,city1,possibleCityOrder)
				possibleCityOrder = np.where(possibleCityOrder==-1,city2,possibleCityOrder)

				# Work out the new distances
				# This can be done more efficiently
				newDistanceTravelled = 0
				for j in range(nCities-1):
					newDistanceTravelled += distances[possibleCityOrder[j],possibleCityOrder[j+1]]
				distanceTravelled += distances[cityOrder[nCities-1],0]

				if newDistanceTravelled < distanceTravelled or (distanceTravelled - newDistanceTravelled) < T*np.log(np.random.rand()):
					distanceTravelled = newDistanceTravelled
					cityOrder = possibleCityOrder

			# Annealing schedule
			T = c*T

	return cityOrder, distanceTravelled

In [ ]:
def runAll():
	import time

	print ("Exhaustive search")
	start = time.time()
	print (exhaustive(distances))
	finish = time.time()
	print (finish-start)

	print ("Greedy search")
	start = time.time()
	print (greedy(distances))
	finish = time.time()
	print (finish-start)

	print ("Hill Climbing")
	start = time.time()
	print (hillClimbing(distances))
	finish = time.time()
	print (finish-start)

	print ("Simulated Annealing")
	start = time.time()
	print (simulatedAnnealing(distances))
	finish = time.time()
	print (finish-start)


In [ ]:
nCities = 6
distances = makeTSP(nCities)

print (distances)

[[0.         0.84651352 1.10140415 1.2145918  0.36861101 1.67713424]
 [0.84651352 0.         1.50883745 0.41167911 0.48176148 0.97978896]
 [1.10140415 1.50883745 0.         1.91151795 1.25933165 1.76146814]
 [1.2145918  0.41167911 1.91151795 0.         0.8473929  0.98813156]
 [0.36861101 0.48176148 1.25933165 0.8473929  0.         1.37612319]
 [1.67713424 0.97978896 1.76146814 0.98813156 1.37612319 0.        ]]


In [ ]:
runAll()

Exhaustive search
((2, 5, 3, 1, 4, 0), 4.0116513043304955)
0.009154796600341797
Greedy search
(array([1, 3, 4, 0, 2, 5]), 6.167689552093941)
0.0016469955444335938
Hill Climbing
(array([2, 5, 3, 1, 4, 0]), 4.0116513043304955)
0.040738821029663086
Simulated Annealing
(array([4, 5, 3, 1, 2, 0]), 5.386175469129828)
0.0024673938751220703


In [ ]:
nCities = 12
distances = makeTSP(nCities)#

runAll()

Exhaustive search
((7, 5, 10, 1, 11, 4, 8, 9, 6, 2, 3, 0), 6.292006101750959)
3034.9405987262726
Greedy search
(array([ 1, 11,  0,  3,  2,  6,  9,  4,  8,  5, 10,  7]), 8.67601202501937)
0.0004563331604003906
Hill Climbing
(array([ 2, 10,  7,  6,  9,  3,  1,  8,  0, 11,  5,  4]), 11.89180794397529)
0.05714535713195801
Simulated Annealing
(array([ 0,  2,  3, 10,  6,  5,  7,  1,  8,  4, 11,  9]), 11.177571852871685)
0.002109527587890625
